# mdsvex

>


In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents,
)

from pathlib import Path
from datetime import datetime
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
source = GitSource(
    name="mdsvex",
    doc_dir=".",
    repo="https://github.com/pngwn/MDsveX",
    commit="main",
    ecosystems=["svelte", "js"],
)

commit, clone_dir = clone_repo(source)
source.commit = commit

66: 5093.0/5093.0 0 8.24 MiB | 2.58 MiB/ss

In [ ]:
run_id = datetime.now().strftime("%Y%m%d-%H%M%S")
virtual_root = settings.project_root / "_tmp_docs" / "mdsvex" / run_id

# Create the structure that will become:
# /mdsvex/
# /mdsvex/mdsvex
# /mdsvex/svast-utils
# /mdsvex/svast
# /mdsvex/svelte-parse
(virtual_root / "mdsvex").mkdir(parents=True, exist_ok=True)

# mdsvex main docs come from the site single-page docs (svtext)
site_docs = (clone_dir / "packages" / "site" / "src" / "routes" / "_docs.svtext").read_text()
(virtual_root / "mdsvex" / "mdsvex.svtext").write_text(site_docs)

# Subpackages are pulled from their READMEs
subpackages = {
    "svast-utils": clone_dir / "packages" / "svast-utils" / "README.md",
    "svast": clone_dir / "packages" / "svast" / "README.md",
    "svelte-parse": clone_dir / "packages" / "svelte-parse" / "README.md",
}

for name, readme_path in subpackages.items():
    (virtual_root / "mdsvex" / f"{name}.md").write_text(readme_path.read_text())

In [ ]:
tree = build_markdown_doc_tree(virtual_root, extensions=[".md", ".svtext"])

In [ ]:
tree.children

[DocItem(origPath=PosixPath('mdsvex/mdsvex.svtext'), name='mdsvex.svtext', displayName='mdsvex.svtext', digest='', short_digest='', essence='', relevant=True, usage=Usage(input=0, output=0, details=None), token_counts=TokenCounts(fulltext=0, digest=0, short_digest=0), fulltext='# mdsvex\n\nmdsvex is a markdown preprocessor for [Svelte][svelte] components. Basically [MDX][mdx] for Svelte.\n\nThis preprocessor allows you to use Svelte components in your markdown, or markdown in your Svelte components.\n\nmdsvex supports all Svelte syntax and _almost_ all markdown syntax. See [limitations](docs/#limitations) for more information.\n\nYou can do this:\n\n```svx\n<script>\n\timport { Chart } from "../components/Chart.svelte";\n</script>\n\n# Here’s a chart\n\nThe chart is rendered inside our MDsveX document.\n\n<Chart />\n```\n\nIt uses [unified], [remark] and [rehype] and you can use any [remark plugins][remark_plugins] or [rehype plugins][rehype_plugins] to enhance your experience.\n\n[Try

In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+45.054s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing mdsvex/mdsvex.svtext
+0.299s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing mdsvex/svast-utils.md
+0.015s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing mdsvex/svast.md
+0.016s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing mdsvex/svelte-parse.md
+30.141s DEBUG preprocessor/nbs/lovely_docs/docs.py:258 llm_process_directory Processing mdsvex


In [ ]:

usage = calculate_total_usage(processed_tree)
cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")
print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")



Total Usage:
  Input tokens: 26,603
  Output tokens: 9,713
  Total tokens: 36,316
Cost:
  Total: $0.08
  Input: $0.03
  Output: $0.05


In [ ]:
processed_tree.children[0].name="mdsvex"
processed_tree.children[0].displayName = "mdsvex"

In [ ]:

save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)